In [0]:
dbutils.fs.mkdirs("dbfs:/delta_lab")
#urls = [("products.csv", "https://raw.githubusercontent.com/MicrosoftLearning/mslearn-databricks/main/data/products.csv")]
dbutils.fs.cp("https://raw.githubusercontent.com/MicrosoftLearning/mslearn-databricks/main/data/products.csv", f"dbfs:/delta_lab/products.csv")


True

In [0]:
df = spark.read.load('/delta_lab/products.csv', format='csv', header=True)
display(df.limit(10))

ProductID,ProductName,Category,ListPrice
771,"Mountain-100 Silver, 38",Mountain Bikes,3399.9900
772,"Mountain-100 Silver, 42",Mountain Bikes,3399.9900
773,"Mountain-100 Silver, 44",Mountain Bikes,3399.9900
774,"Mountain-100 Silver, 48",Mountain Bikes,3399.9900
775,"Mountain-100 Black, 38",Mountain Bikes,3374.9900
776,"Mountain-100 Black, 42",Mountain Bikes,3374.9900
777,"Mountain-100 Black, 44",Mountain Bikes,3374.9900
778,"Mountain-100 Black, 48",Mountain Bikes,3374.9900
779,"Mountain-200 Silver, 38",Mountain Bikes,2319.9900
780,"Mountain-200 Silver, 42",Mountain Bikes,2319.9900


In [0]:
delta_table_path = "/delta/products-delta"
df.write.mode("overwrite").format("delta").save(delta_table_path)

In [0]:
 %sh
 ls /dbfs/delta/products-delta

_delta_log
deletion_vector_95b137d6-daf6-4ea5-8b3f-e0e418e5121c.bin
part-00000-6453a694-901e-4cc8-bf45-1ab245085795.c000.snappy.parquet
part-00000-e711f44c-50c6-487c-8f55-490a1beb6c3f.c000.snappy.parquet
part-00000-f1fae48f-c5e0-49fb-ba95-2e0a100c3efe.c000.snappy.parquet
part-00000-f2cdb9c8-e162-45c4-bbac-e0444fe39406.c000.snappy.parquet


In [0]:
from delta.tables import *
from pyspark.sql.functions import *
   
# Create a deltaTable object
deltaTable = DeltaTable.forPath(spark, delta_table_path)
# Update the table (reduce price of product 771 by 10%)
deltaTable.update(
    condition = "ProductID == 771",
    set = { "ListPrice": "ListPrice * 0.9" })
# View the updated data as a dataframe
deltaTable.toDF().show(10)

+---------+--------------------+--------------+---------+
|ProductID|         ProductName|      Category|ListPrice|
+---------+--------------------+--------------+---------+
|      772|Mountain-100 Silv...|Mountain Bikes|3399.9900|
|      773|Mountain-100 Silv...|Mountain Bikes|3399.9900|
|      774|Mountain-100 Silv...|Mountain Bikes|3399.9900|
|      775|Mountain-100 Blac...|Mountain Bikes|3374.9900|
|      776|Mountain-100 Blac...|Mountain Bikes|3374.9900|
|      777|Mountain-100 Blac...|Mountain Bikes|3374.9900|
|      778|Mountain-100 Blac...|Mountain Bikes|3374.9900|
|      779|Mountain-200 Silv...|Mountain Bikes|2319.9900|
|      780|Mountain-200 Silv...|Mountain Bikes|2319.9900|
|      781|Mountain-200 Silv...|Mountain Bikes|2319.9900|
+---------+--------------------+--------------+---------+
only showing top 10 rows


In [0]:
new_df = spark.read.format("delta").load(delta_table_path)
new_df.show(10)

+---------+--------------------+--------------+---------+
|ProductID|         ProductName|      Category|ListPrice|
+---------+--------------------+--------------+---------+
|      772|Mountain-100 Silv...|Mountain Bikes|3399.9900|
|      773|Mountain-100 Silv...|Mountain Bikes|3399.9900|
|      774|Mountain-100 Silv...|Mountain Bikes|3399.9900|
|      775|Mountain-100 Blac...|Mountain Bikes|3374.9900|
|      776|Mountain-100 Blac...|Mountain Bikes|3374.9900|
|      777|Mountain-100 Blac...|Mountain Bikes|3374.9900|
|      778|Mountain-100 Blac...|Mountain Bikes|3374.9900|
|      779|Mountain-200 Silv...|Mountain Bikes|2319.9900|
|      780|Mountain-200 Silv...|Mountain Bikes|2319.9900|
|      781|Mountain-200 Silv...|Mountain Bikes|2319.9900|
+---------+--------------------+--------------+---------+
only showing top 10 rows


In [0]:
new_df = spark.read.format("delta").option("versionAsOf", 0).load(delta_table_path)
new_df.show(10)

+---------+--------------------+--------------+---------+
|ProductID|         ProductName|      Category|ListPrice|
+---------+--------------------+--------------+---------+
|      771|Mountain-100 Silv...|Mountain Bikes|3399.9900|
|      772|Mountain-100 Silv...|Mountain Bikes|3399.9900|
|      773|Mountain-100 Silv...|Mountain Bikes|3399.9900|
|      774|Mountain-100 Silv...|Mountain Bikes|3399.9900|
|      775|Mountain-100 Blac...|Mountain Bikes|3374.9900|
|      776|Mountain-100 Blac...|Mountain Bikes|3374.9900|
|      777|Mountain-100 Blac...|Mountain Bikes|3374.9900|
|      778|Mountain-100 Blac...|Mountain Bikes|3374.9900|
|      779|Mountain-200 Silv...|Mountain Bikes|2319.9900|
|      780|Mountain-200 Silv...|Mountain Bikes|2319.9900|
+---------+--------------------+--------------+---------+
only showing top 10 rows


In [0]:
deltaTable.history(10).show(10, False, True)

-RECORD 0-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 version             | 4                                                                                                                                                                                                                                                                                                                            
 timestamp           | 2025-11-12 14:35:59                                                                                                                                                                                                                                                                                    

In [0]:
new_df.write.format("delta").mode("overwrite").saveAsTable("AdventureWorks.ProductsManaged")
spark.sql("DESCRIBE EXTENDED AdventureWorks.productsmanaged").show(truncate=False)

+----------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------+
|col_name                    |data_type                                                                                                                                                                                          |comment|
+----------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------+
|ProductID                   |string                                                                                                                                                                                             |NULL   |
|ProductName                 |string                        

In [0]:
%sql
SELECT
  `Category`,
  AVG(CAST(`ListPrice` AS DOUBLE)) AS `AverageListPrice`
FROM
  `dbworkspace`.`adventureworks`.`productsmanaged`
GROUP BY
  `Category`;

Category,AverageListPrice
Headsets,87.07333333333334
Wheels,220.92928571428567
Bottom Brackets,92.24
Touring Frames,631.4155555555556
Mountain Bikes,1683.3649999999982
Pedals,64.01857142857143
Derailleurs,106.475
Chains,20.24
Pumps,22.49
Hydration Packs,54.99
